In [1]:
import openmm as mm
import numpy as np
import openmm.app as app
import simtk.unit as unit
from openmmforcefields.generators import GAFFTemplateGenerator as gen
from openff.toolkit import Topology
import pytraj as pt
import os
import matplotlib.pyplot as plt

from openmm.app import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet



In [2]:
base = '/scratch/htc/fsafarov/structures/8ef5_july_2025/8ef5'

#We use psf and crd system because pdb files does not support the 6-digit base system for ATOM ID's, which is in our case utilized for POPC. 
#Input below is from psf and crd from CHARMM-GUI.

psf = CharmmPsfFile(os.path.join(base, 'step5_assembly.psf'))
crd = CharmmCrdFile(os.path.join(base, 'step5_assembly.crd'))


/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.27; phase=180.0; per=2> with <DihedralType: k=0.03; phase=180.0; per=2>
  warnings.warn('Replacing dihedral %r with %r' %
/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.0; phase=0.0; per=3> with <DihedralType: k=0.12; phase=0.0; per=3>
  warnings.warn('Replacing dihedral %r with %r' %


In [ ]:
with open('/scratch/htc/fsafarov/states/mor_simulation_1/system_npgt_2.xml') as input:
    system = mm.XmlSerializer.deserialize(input.read())

In [ ]:
state = mm.XmlSerializer.deserialize(open('/scratch/htc/fsafarov/states/mor_simulation_1/npgt_eq_2.xml').read())
centered_positions = state.getPositions()

In [14]:
integrator_eq = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picoseconds, 2.0*unit.femtoseconds)

In [ ]:
platform = mm.Platform.getPlatformByName('CUDA')
#to run the simulation on several GPU's parallelly
properties  = {'CudaDeviceIndex': '0,1,2', 'CudaPrecision': 'mixed'}

In [ ]:
#loads checkpoint if exists
simulation_eq = app.Simulation(psf.topology, system, integrator_eq, platform, properties)
simulation_eq.loadCheckpoint('/scratch/htc/fsafarov/traj/mor_simulation_suru_prot/checkpoint_pr_npgt_2.chk')
state_1 = simulation_eq.context.getState(getVelocities=True)
velocities = state_1.getVelocities(asNumpy = True)


In [ ]:
for i in reversed(range(system.getNumForces())):
    force = system.getForce(i)
    if isinstance(force, mm.MonteCarloMembraneBarostat) or isinstance(force, mm.CustomExternalForce):
        system.removeForce(i)

In [ ]:
for i in range(system.getNumForces()):
    force = system.getForce(i)
    print(force)

In [ ]:
protein_restraint = mm.CustomExternalForce('k_2*periodicdistance(x, y, z, x0, y0, z0)^2')
# system.addForce(restraint)non-standard (HET) residues (excluding water).
# - Inter-residue connectivity of HET  groups to standard groups (including water) or to other HET groups. 
# - Disulfide bridges specified in the  SSBOND records have corresponding records.

protein_restraint.addGlobalParameter('k_2', 5*unit.kilocalories_per_mole/unit.angstrom**2)
protein_restraint.addPerParticleParameter('x0')
protein_restraint.addPerParticleParameter('y0')
protein_restraint.addPerParticleParameter('z0')
system.addForce(protein_restraint)



In [ ]:
#restrain the protein and ligand
std_amino_acids = ['GLY', 'TYR', 'PHE', 'ARG', 'HIS', 'ALA', 'PRO', 'GLU', 'SER', 'LYS',
    'THR', 'MET', 'CYS', 'LEU', 'GLN', 'ASN', 'VAL', 'ILE', 'ASP', 'TRP']

for residue in psf.topology.residues():
    if residue.name in std_amino_acids:
        for atom in residue.atoms():
            if atom.name in ['N', 'CA', 'C', 'O']:
                pos = centered_positions[atom.index]
                protein_restraint.addParticle(atom.index, [pos[0], pos[1], pos[2]])

if residue.name == '7V7':
    for atom in residue.atoms():
        pos = centered_positions[atom.index]
        protein_restraint.addParticle(atom.index, [pos[0], pos[1], pos[2]])
        
    

In [ ]:
simulation_eq.context.reinitialize()
simulation_eq.context.setPositions(centered_positions)
simulation_eq.context.setVelocities(velocities)

In [ ]:
a_vec, b_vec, c_vec = state.getPeriodicBoxVectors()

simulation_eq.context.setPeriodicBoxVectors(a_vec,b_vec,c_vec)

In [ ]:
dict(simulation_eq.context.getParameters()) #check the padrameters!!!

In [ ]:
#check the forces
state_2 = simulation_eq.context.getState(getForces=True, getEnergy=True)
forces = state_2.getForces(asNumpy = True).value_in_unit(unit.kilojoule/unit.nanometer/unit.mole)
norm = np.linalg.norm(forces, axis = 1)
max_force = max(norm)
np.argmax(norm), np.max(norm)

In [ ]:
#check energy
print("Potential energy:", state_2.getPotentialEnergy())
print("Kinetic energy:", state_2.getKineticEnergy())

In [27]:
# simulation_eq.reporters.append(app.StateDataReporter('/scratch/htc/fsafarov/traj/output_membrane_barostat_2.log', 1000, step=True, potentialEnergy=True, temperature=True))

simulation_eq.reporters.append(app.DCDReporter('/scratch/htc/fsafarov/traj/mor_simulation_suru_prot/trajectory_pr_nvt_final_1.dcd', 500, enforcePeriodicBox=True))
simulation_eq.reporters.append(app.CheckpointReporter('/scratch/htc/fsafarov/traj/mor_simulation_suru_prot/checkpoint_pr_nvt_final_1.chk', 500))
simulation_eq.step(25000) #


In [ ]:
#NVT pt.2
simulation_eq.context.setParameter('k_2', 0.0*unit.kilocalories_per_mole/unit.angstrom**2)
integrator_eq.setTemperature(310.15*unit.kelvin)
simulation_eq.context.setVelocitiesToTemperature(310.15*unit.kelvin)

dict(simulation_eq.context.getParameters()) #check the padrameters!!!

In [ ]:
#check the forces
state_3 = simulation_eq.context.getState(getForces=True, getEnergy=True)
forces = state_3.getForces(asNumpy = True).value_in_unit(unit.kilojoule/unit.nanometer/unit.mole)
norm = np.linalg.norm(forces, axis = 1)
max_force = max(norm)
np.argmax(norm), np.max(norm)

In [ ]:
#check energy
print("Potential energy:", state_3.getPotentialEnergy())
print("Kinetic energy:", state_3.getKineticEnergy())

In [ ]:
# simulation_eq.reporters.append(app.StateDataReporter('', 1000, step=True, potentialEnergy=True, temperature=True))

simulation_eq.reporters.append(app.DCDReporter('/scratch/htc/fsafarov/traj/mor_simulation_suru_prot/trajectory_pr_nvt_final_2.dcd', 500, enforcePeriodicBox=True))
simulation_eq.reporters.append(app.CheckpointReporter('/scratch/htc/fsafarov/traj/mor_simulation_suru_prot/checkpoint_pr_nvt_final_2.chk', 500))
simulation_eq.step(25000) #


In [ ]:
simulation_eq.saveState('/scratch/htc/fsafarov/states/mor_simulation_1/nvt_eq_2.xml')

In [ ]:
with open('/scratch/htc/fsafarov/states/mor_simulation_1/system_nvt_2.xml', 'w') as f:
    f.write(mm.XmlSerializer.serialize(system))